<a href="https://colab.research.google.com/github/rreimche/genai-exam/blob/main/LitRev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites:

- Google Colab
- Access to a Google Colab Secret named "HF_API_TOKEN" containing a Huggingface API token with read access.

In [ ]:
#@title Settings
from google.colab import userdata

download_dir = "papers"  # directory name to download papers to revie to
huggingface_apikey = userdata.get('HF_API_TOKEN')  # use colab secrets to store the huggingface api key
groq_key = userdata.get('GROQ_KEY')  #  use colab secrets to store the huggingface api key


In [ ]:
#@title Declare bibtex references and parse them into a variable for later usage


bibtext_references = """


@misc{qian2024chatdevcommunicativeagentssoftware,
      title={ChatDev: Communicative Agents for Software Development},
      author={Chen Qian and Wei Liu and Hongzhang Liu and Nuo Chen and Yufan Dang and Jiahao Li and Cheng Yang and Weize Chen and Yusheng Su and Xin Cong and Juyuan Xu and Dahai Li and Zhiyuan Liu and Maosong Sun},
      year={2024},
      eprint={2307.07924},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2307.07924},
},

@misc{nguyen2024agilecoderdynamiccollaborativeagents,
      title={AgileCoder: Dynamic Collaborative Agents for Software Development based on Agile Methodology},
      author={Minh Huynh Nguyen and Thang Phan Chau and Phong X. Nguyen and Nghi D. Q. Bui},
      year={2024},
      eprint={2406.11912},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2406.11912},
},


"""



In [ ]:
#@title Install dependencies
!pip install -q autogen arxiv scholarly crossrefapi beautifulsoup4 requests cloudscraper pymupdf nltk autogen-agentchat autogen-ext[openai] groq pymupdf;
!pip install -q --pre bibtexparser;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.5/642.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.0 MB/s eta 0:00:00
   ━━━━

In [ ]:
#@title Preparation for agents: model client connections
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool
from autogen_core.models import UserMessage
#from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# INSTANTIATE MODEL CLIENTS

# This client will be used for paper summarization
text_model_client = OpenAIChatCompletionClient(
    #model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    #model="facebook/bart-large-xsum",
    model="mistralai/Mistral-7B-Instruct-v0.3",
    #model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    base_url="https://router.huggingface.co/hf-inference/v1",
    api_key=huggingface_apikey,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# We need another inference point for downloader agent,
# because huggingface can't serve reflection on tool use
# so that autogen agent understands it
tool_model_client = OpenAIChatCompletionClient(
    #model="llama3-70b-8192",
    model="llama-3.3-70b-versatile",
    #model="deepseek-r1-distill-qwen-32b",
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)


# CHECK CONNECTION

async def check_model_connect() -> None:
    messages = [
        UserMessage(content="What is the capital of France?", source="user"),
    ]
    response = await text_model_client.create(messages=messages)
    response_downloader = await tool_model_client.create(messages=messages)

    print(response.content)
    print(response_downloader.content)

await check_model_connect()

The capital of France is Paris. It is also one of the most famous cities in the world, known for its architecture, art, fashion, and cuisine. Paris is home to many iconic landmarks, such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.
The capital of France is Paris.


In [ ]:
#@title Downloader agent
import arxiv
import os
import datetime
import json
import sys
from dataclasses import dataclass
from autogen_core.models import ChatCompletionClient
from autogen_core import message_handler

# HELPER AND TOOL FUNCTIONS

def maybe_create_dir(donwload_dir: str) -> None:
    """
        A helper function to create the downloads directory
        in colab root if the directory is not yet present.
    """

    try:
        os.makedirs(donwload_dir, exist_ok=True)
        #print(f"Directory '{download_dir}' created successfully.")
    except OSError as e:
        raise Exception(f"Error creating directory '{download_dir}': {e}")

def download_paper(eprint_id: str) -> str:
    """
        Receives a eprint_id of a paper from arxiv, downloads the specified paper
        and saves it in downloads directory.
        The filepath of the downloaded paper is in the resulting dictionary
        with the key "filepath".

        :param eprint_id: string, the arxiv eprint_id.
        :return: string with filepath of the downloaded paper.
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    result = {
        "eprint_id": eprint_id,
        "success": False,
        "filepath": None,
        }


    if eprint_id:
        #print(f"{datetime.datetime.now()} - Trying arXiv for: {eprint_id}")
        try:
            client = arxiv.Client()
            search = arxiv.Search(id_list=[eprint_id], max_results=1)

            #arxiv_result = next(search.results(), None)
            arxiv_result = next(client.results(search))
            if arxiv_result:
                #print(f"  Found on arXiv: {arxiv_result.title}")
                maybe_create_dir(download_dir)
                filepath = os.path.join(download_dir, f"{arxiv_result.title.replace(' ', '_')}.pdf")
                arxiv_result.download_pdf(filename=filepath)
                result["success"] = True
                result["filepath"] = filepath
                #print(f"  Download successful: {filepath}")
            else:
                # print(f"  No arXiv result found for ID {eprint_id}.")
                raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")
                #return f"ERROR: No arXiv result found for ID {eprint_id}."

        except Exception as e:
            #print(f"arXiv download failed for: {eprint_id}: {e}")
            raise Exception(f"ERROR: arXiv download failed for: {eprint_id}: {e}")
            #return f"ERROR: arXiv download failed for: {eprint_id}: {e}"

    # Check if the download was NOT successful
    if not result["success"]:
        #print(f"{datetime.datetime.now()} - Download failed or not attempted for: {eprint_id}")
        raise Exception(f"ERROR: Download failed or not attempted for: {eprint_id}")
        #return f"ERROR: Download failed or not attempted for: {eprint_id}"

    return result["filepath"]


# DOWNLOADER AGENT WITH TOOL

def create_downloader(given_name: str) -> AssistantAgent:
    downloader_tool = FunctionTool(download_paper, description="Given a string denoting an eprint_id of a page on arxiv, downloads a paper")

    return AssistantAgent(
        name=given_name,
        description="An agent that uses downloader tool to download a paper from Arxiv identified by a eprint_id",
        model_client=tool_model_client,
        tools=[downloader_tool],
        reflect_on_tool_use=True,
        system_message="""
            Use downloader tool to download specified paper(s) from arxiv.
            Here is an example of how you must respond in success case:
              'mypapers/veryinterstingpaper.pdf'

            Very important: You MUST return the full filepath of the downloaded
            paper EXACTLY as it was given by the downloader tool.

            If download was not successful, respond like this:
              'ERROR: Download for paper with eprint id XXX failed for the following reason: REASON'.
          """,
    )

# CHECK IF DOWNLOAD WORKS

async def downloader_run() -> None:
    response = await create_downloader("Test").on_messages(
        [TextMessage(content="Download the paper with eprint_id 2402.01411", source="user")],
        cancellation_token=CancellationToken(),
    )


    #print(response.inner_messages)
    #print(response.chat_message)
    print(response.chat_message.content)



# Use asyncio.run(assistant_run()) when running in a script.
#await downloader_run()


In [ ]:
#@title Agents to get paper text: Informant and Parser
import arxiv
from typing import Dict
import pymupdf
import requests
from autogen_agentchat.ui import Console

def get_arxiv_data(eprint_id: str) -> Dict[str, str]:
    """
    Given arxiv eprint_id of a scientific paper, returns title of the paper
    along with the PDF URL.

    :param eprint_id: string, the arxiv eprint_id.
    :return: dictionary with keys "title" and "pdf_url".
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    client = arxiv.Client()
    search = arxiv.Search(id_list=[eprint_id], max_results=1)

    try:

        arxiv_result = next(client.results(search))
        if arxiv_result:
            return {
                "title": arxiv_result.title,
                "pdf_url": arxiv_result.pdf_url,
            }
        else:
            raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")

    except Exception as e:
        raise Exception(f"ERROR: getting data from arxiv failed for: {eprint_id}: {e}")


def parse_paper(url: str) -> str:
    """
    Given URL of a PDF as a string, returns the contents as a string

    :param url: string, the URL of the PDF.
    :return: string with the contents of the PDF.
    """

    try:

        r = requests.get(url)
        data = r.content
        doc = pymupdf.Document(stream=data)
        text = chr(12).join([page.get_text() for page in doc])
        return text

    except Exception as e:
        # print(f"Error parsing {title} ({filepath}): {e}")
        raise Exception(f"ERROR: Error parsing the paper at {url}: {e}")


async def get_concept(eprint_id: str) -> str:
    """
    Takes a eprint id from arxiv and returns the summarization of the related paper.

    :param eprint_id: string, the arxiv eprint_id.
    :return: string with PDF contents.
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"



    try:
        arxiv_data = get_arxiv_data(eprint_id)
        #print(arxiv_data["pdf_url"])
        text = parse_paper(arxiv_data["pdf_url"])

        concept_prompt = f"""
          You are a professor of computer science specialized in multi agent systems
          and generative AI. Your task ist to summarize a certain scientific paper.

          Here is the text of the paper:

          -------start of paper--------
          {text}
          -------end of paper--------

          Use the provided text to write the summarization.
          Write consice and clear using the highest academic standards.
          It is very important to provide short descriptions of key contributions.
          Fit your answer in 500 words.


    """

        concept = await text_model_client.create([UserMessage(content=concept_prompt, source="user")])
        #print(concept)
    except Exception as e:
        raise Exception(f"ERROR: Error downloading, parsing or summarizing the paper identified by eprint id {eprint_id}: {e}")

    return concept


tool_getsummary = FunctionTool(
    get_concept,
    description = "Takes a eprint id of a paper published on arxiv and returns a summarization of the related paper."
)


paper_summarizer = AssistantAgent(
    name="parser",
    description="Provided with arxiv eprint id, returns a summarization of the related paper",
    model_client=tool_model_client,
    tools=[tool_getsummary],
    reflect_on_tool_use=True,
    system_message="""
        You are a a professor of computer science. Use eprint ids of papers
        published at Arxiv to use the tool you're equipped with (tool_getsummary),
        which will give you the summarization of the related paper.
        Give the text of the paper back as your answer. Answer concise
        and structured.

        You are not allowed to do any other work like planning, writing summarizations or reviews.
    """

)

# CHECK IF WORKS

async def get_paper_summary_run() -> None:
    #response_i = await paper_informant.on_messages(
    #    [TextMessage(content="Give me the infos of the paper with eprint_id 2307.07924", source="user")],
    #    cancellation_token=CancellationToken(),
    #)

    #prev_answer = response_i.chat_message.content

    #print(prev_answer)

    #response_p = await paper_parser.on_messages(
    #    [TextMessage(content=f"Give me the text of paper, which url is specified in the following text: {prev_answer}", source="user")],
    #    cancellation_token=CancellationToken(),
    #)

    # for deeper debugging
    #await Console(
    #    paper_conceptualizer.on_messages_stream(
    #        [TextMessage(content=f"Give me the concept of a paper with eprint_id 2307.07924", source="user")],
    #        cancellation_token=CancellationToken(),
    #    ),
    #    output_stats=True,  # Enable stats printing.
    #)

    response = await paper_summarizer.on_messages(
        [TextMessage(content="Give me a summarization of the paper with eprint_id 2307.07924", source="user")],
        cancellation_token=CancellationToken(),
    )

    #print(response.inner_messages)
    #print(response.chat_message)
    #print(response.chat_message.content)

    return response.chat_message.content





# Use asyncio.run(assistant_run()) when running in a script.
test_paper_summary = await get_paper_summary_run()
print(test_paper_summary)




The paper with eprint_id 2307.07924 introduces ChatDev, a software development framework that utilizes large language models and multiprovised agents to streamline the development process. ChatDev offers various auxiliary roles, including a requirements analyst, a professional programmer, and a tester, who perform their functions in a designated phase. The core of ChatDev lies in the chat chain, which segments each phase into smaller subtasks, enabling multi-turn dialogues between the agents to collaboratively propose and build solutions. The framework also employs a mechanism called communicative dehallucination to minimize "coding hallucinations," wherein the assistant actively seeks more detailed information before providing responses to avoid generating incomplete or inconsistent code. The use of ChatDev improves various performance metrics, including completeness, executability, and consistency, and is superior to related LLM-based software development methods.


In [ ]:
#@title Reviewer agent
from typing import List


reviewer = AssistantAgent(
    name="reviewer",
    description="""
        Given a several summarizations (short description of important paper contents)
        for several scientific papers, writes a scientific literature review.
    """,
    model_client=text_model_client,
    #system_message="""
#        You are a professor of computer science preparing a review
#        of recent publications on the topic of multi-agent llm-based systems
#        for end-to-end software development (such systems receive requirements
#        as input and deliver a ready system as output with optional system-user
#        interactions inbetween). You will receive the conceptualisations of
#        several papers (that is, consice enumeration of important contributions of the papers)
#        and write a concise and factually detailed literature review.
#        Stick to scientific standards regarding style and rigour.
#        You must fit your review in 500 words.

#        It is very important to depict the key contributions of the papers.
#""",
    system_message="""
        You are a professor of computer science preparing a review
        of recent publications on the topic of multi-agent llm-based systems
        for end-to-end software development (such systems receive requirements
        as input and deliver a ready system as output with optional system-user
        interactions inbetween). Use provided conceptualisations of
        several papers (that is, consice enumeration of important contributions of the papers)
        and write a concise and factually detailed literature review.
        Stick to scientific standards regarding style and rigour, but provide
        only the review text, omitting authors, refences and other metainformation.
        You must fit your review in 500 words.

        It is very important to depict the key contributions of the papers.

        You do not do any other work like downloading, parsing, writing conceptualizations or summarizations.
""",
)


# CHECK IF REVIEWING WORKS

async def reviewer_run(summarizations: List[str]) -> str:

    prompt_rev = f"Here are {len(summarizations)} paper summarizations that you need to review:\n"
    for summarization in summarizations:
        prompt_rev += f"------start summarizations-------\n{summarization}\n------end summarizations-------\n"
    prompt_rev += "Write the scientific literature review."


    response = await reviewer.on_messages(
       [TextMessage(content=prompt_rev, source="user")],
       cancellation_token=CancellationToken(),
    )

    #print(paper_text)
    #print(response.inner_messages)
    #print(response.chat_message.models_usage)
    return response.chat_message.content



# Use asyncio.run(assistant_run()) when running in a script.
review = await reviewer_run([test_paper_summary, test_paper_summary])
print(review)


In this literature review, we focus on two recent publications that propose the use of multi-agentllm-based systems for end-to-end software development. The first paper, with eprint_id 2307.07924, introduces ChatDev, a novel software development framework. ChatDev leverages large language models and multiprovider agents to streamline the development process, consisting of auxiliary roles such as a requirements analyst, professional programmer, and tester, each performing their functions in a designated phase.

The core of ChatDev is its chat chain, which segments each phase into smaller subtasks, enabling multi-turn dialogues between the agents to collaboratively propose and build solutions. To minimize "coding hallucinations," a common issue in such systems, ChatDev employs a mechanism called communicative dehallucination. This mechanism empowers the assistant to actively seek more detailed information before providing responses, thereby reducing the generation of incomplete or incons

In [ ]:
#@title Planner

planner = AssistantAgent(
    name="planner",
    description="Plans the literature review process, assigns tasks to other agents.",
    model_client=tool_model_client,
    system_message="""
      Your task is to plan scientific review process for given papers, assigning tasks to agents.
      You have several agents in your disposition:
      1. Informant: given a eprint_id of a paper published on arxiv,
      gets the paper metadata, including title and PDF URL.
      2. Parser: given a singe URL of a paper PDF file, gets the file, parses it and returns the text.
      3. Conceptualizer: given a single text of a scientific paper, this agent will write
      a conceptualization -- structure and key contributions delivered by the paper.
      4. Summarizer: given a single conceptualization of a paper (title, structure of
      the paper, key points and contributions), writes a summarization like a paper abstract.
      5. Reviewer: given several summarizations (short description of important paper contents)
      for several scientific papers, writes a scientific literature review on these papers.

      When assigning tasks, use this format:
      1. <agent> : <task>

      Only plan the work of others, do not do any work yourself.
    """
)

# Check planner
async def planner_run(task: str) -> None:


    await Console(
        planner.on_messages_stream(
            [TextMessage(content=task, source="user")],
            cancellation_token=CancellationToken(),
        ),
        output_stats=True,
    )

    #print(paper_text)
    #print(response.inner_messages)
    #print(response.chat_message.models_usage)

await planner_run("Write a literature review on the papers, identified by the following arxiv eprint ids: 2307.07924, 2406.11912")

---------- planner ----------
To generate a literature review on the given papers, the following tasks should be assigned to the agents:

1. Informant : Get the paper metadata for eprint_id 2307.07924
2. Informant : Get the paper metadata for eprint_id 2406.11912
3. Parser : Parse the PDF file from the URL provided by the Informant for eprint_id 2307.07924
4. Parser : Parse the PDF file from the URL provided by the Informant for eprint_id 2406.11912
5. Conceptualizer : Conceptualize the text parsed by the Parser for eprint_id 2307.07924
6. Conceptualizer : Conceptualize the text parsed by the Parser for eprint_id 2406.11912
7. Summarizer : Summarize the conceptualization for eprint_id 2307.07924
8. Summarizer : Summarize the conceptualization for eprint_id 2406.11912
9. Reviewer : Write a scientific literature review using the summarizations for both eprint_id 2307.07924 and 2406.11912

After these tasks are completed, the Reviewer will have generated a literature review on the papers 

In [ ]:
#@title Preparing the task for the system
from bibtexparser.bparser import BibTexParser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
from bibtexparser.customization import convert_to_unicode
from typing import List


def parse_bibtex_string(bibtex_string):

    if not bibtex_string:
        return []  # Handle empty input

    try:
        parser = BibTexParser()
        parser.customization = convert_to_unicode
        parser.ignore_nonstandard_strings = True  # Avoid errors with non-standard fields
        db = parser.parse(bibtex_string)
        return db.entries
    except Exception as e:
        print(f"Error parsing BibTeX string: {e}")
        return None  # Indicate parsing failure

def check_parsed_bibtex(items: List[dict])  -> None:
    for entry in bibtex_items:
        print(f"Entry Type: {entry['ENTRYTYPE']}")
        print(f"  Key: {entry['ID']}")
        for key, value in entry.items():
            if key not in ['ENTRYTYPE', 'ID']:
                print(f"  {key}: {value}")
        print("-" * 20)


bibtex_items = parse_bibtex_string(bibtext_references)

#check_parsed_bibtex(bibtex_items)

eprints = ", ".join([item['eprint'] for item in bibtex_items])

task = f"Write a literature review on the papers, identified by the following arxiv eprint ids: {eprints}"

print(f"THE TASK\n{task}")

THE TASK
Write a literature review on the papers, identified by the following arxiv eprint ids: 2307.07924, 2406.11912


In [ ]:
#@title Putting it all together
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console




text_mention_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages=5)  # 4 phases x 6 papers + reviewer + user feedback + a bit of slack
termination = text_mention_termination | max_messages_termination

selector_prompt = """
  Select an agent to perform the given task. For this, you have the following
  agent roles:

  {roles}

  It is important to consider conversation context:
  {history}

  Read the above conversation, then select an agent from {participants} to perform the next task.
  Make sure the planner agent has assigned tasks before other agents start working.
  Only select one agent.
"""

team = SelectorGroupChat(
    [planner, paper_informant, paper_parser, conceptualizer, summarizer, reviewer],
    model_client=tool_model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)



In [ ]:
#@title Showtime




await Console(team.run_stream(task=task))

---------- user ----------
Write a literature review on the papers, identified by the following arxiv eprint ids: 2307.07924, 2406.11912
---------- planner ----------
To write a literature review, the following tasks should be assigned to the agents:

1. **Informant**: Get the paper metadata for eprint_id 2307.07924
2. **Informant**: Get the paper metadata for eprint_id 2406.11912
3. **Parser**: Parse the PDF file from the URL provided by the Informant for eprint_id 2307.07924
4. **Parser**: Parse the PDF file from the URL provided by the Informant for eprint_id 2406.11912
5. **Conceptualizer**: Conceptualize the text parsed by the Parser for eprint_id 2307.07924
6. **Conceptualizer**: Conceptualize the text parsed by the Parser for eprint_id 2406.11912
7. **Summarizer**: Summarize the conceptualization for eprint_id 2307.07924
8. **Summarizer**: Summarize the conceptualization for eprint_id 2406.11912
9. **Reviewer**: Write a scientific literature review using the summarizations for b